In [1]:
import pandas as pd 

In [2]:
df_case = pd.read_pickle('df_cases_200906.gzip')
df_label = pd.read_pickle('df_label_200906.gzip')

In [3]:
df_case.head(2)

,CaseId,FileName,Language,StartDate,DocumentType,IsExecuted,OcrText,QualityScore
0,003061226227,003061226227_12045631_Order form_978-1-67767-3...,EN,20191104,Order form,False,Pressure style response character.,0.649292
1,003061226383,003061226383_63912371_Contract Info Pack_978-0...,EN,20190805,Contract Info Pack,False,Soon especially boy thousand traditional.,0.873038


In [4]:
# split dataframe into valid and invalid dataframes
df_invalid=df_case[(df_case['IsExecuted']==False) | (df_case['QualityScore'] <0.81)]
df_valid = df_case[(df_case['IsExecuted']==True) &(df_case['QualityScore'] >=0.81)]

In [5]:
# groupby id , taking list of the filename
grp_invalid = df_invalid.groupby(['CaseId']).agg({'FileName': list}).reset_index()

In [6]:
# groupby id , taking list of the filename
grp_valid = df_valid.groupby(['CaseId']).agg({'FileName': list,'OcrText':list}).reset_index()

In [7]:
# rename the filename of each dataframe as expected
grp_invalid =grp_invalid.rename(columns={"FileName": "InvalidFileNames"})
grp_valid =grp_valid.rename(columns={"FileName": "ValidFileNames"})

In [8]:
# outer join two dataframes
df_final=grp_invalid.merge(grp_valid,on='CaseId',how = 'outer')

In [9]:
# convert NaN to []
import numpy as np
df_final['InvalidFileNames'] = df_final['InvalidFileNames'].apply(lambda x: [] if x is np.nan else x)
df_final['ValidFileNames'] = df_final['ValidFileNames'].apply(lambda x: [] if x is np.nan else x)
df_final['OcrText'] = df_final['OcrText'].apply(lambda x: [] if x is np.nan else x)

In [10]:
# convert list of strings to string
df_final['OcrText'] = df_final['OcrText'].apply(lambda x: ' '.join(x))

In [11]:
df_final= df_final.merge(df_label,on='CaseId',how ='left')

In [12]:
df_final.head(5)

,CaseId,InvalidFileNames,ValidFileNames,OcrText,label_1,label_2
0,003061189229,[003061189229_69115288_Master contract_978-0-1...,[003061189229_21094545_other documents_978-1-8...,Accept sell leader herself if.,False,False
1,003061189242,[003061189242_91658807_other documents_978-0-7...,[],,False,False
2,003061189279,[003061189279_38849442_other documents_978-0-4...,[],,False,False
3,003061189282,[003061189282_21176036_Contract Documents_978-...,[],,False,False
4,003061189333,[003061189333_93380638_Order form_978-1-948186...,[],,False,False


In [13]:
df_final.to_pickle('df_final.gzip')